# Tree Census Final Project

## Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, precision_recall_fscore_support, make_scorer
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from scipy import stats

## Exlporatory Data Analysis and Visualization

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
# load in data with relative path
tree_df = pd.read_csv('2015_Street_Tree_Census_-_Tree_Data.csv')

In [ ]:
# plot missing values by columns
vals = (tree_df.isna().sum() * 100 / len(tree_df)).values
keys = tree_df.columns
plt.tight_layout()
plt.bar(keys, vals)
plt.xlabel("Columns")
plt.ylabel("Percent missing")
plt.xticks(rotation=90, fontsize=8)
plt.title("Percent missing by column")
plt.show()

In [ ]:
# plot NA values by health status
tree_df[['health', 'status']].value_counts(dropna=False).plot(kind='bar')
plt.title('Trees with NaN health')
plt.ylabel("Number of trees")
plt.show()

In [ ]:
# drop dead trees that appear as NA
tree_df_clean = tree_df[tree_df['health'].notna()]
print("Dead or stumps:", tree_df.shape[0] - tree_df_clean.shape[0])

In [ ]:
# other NA values
vals = (tree_df_clean.isna().sum() * 100 / len(tree_df_clean)).values
keys = tree_df_clean.columns
plt.tight_layout()
plt.bar(keys, vals)
plt.xlabel("Columns")
plt.ylabel("Percent missing")
plt.xticks(rotation=90, fontsize=8)
plt.title("Percent missing by column without dead trees")
plt.show()

In [ ]:
'''
Dropped columns:
- borocode and boro_ct, equivalent to borough
- x_sp and y_sp - use latitude and longitude instead
- state - identical for all
- nta_name, equivalent to nta
- zip_city, directly correlated to postcode
- address, estimated from latitude and longitude
- spc_latin, equivalent to spc_common
- created_at, not helpful (time data was added)
- tree_id, not helpful (unique identifier)
- block_id, not helpful (geo location)
- user_type, not helpful (which volunteer analyzed it)
- bin, not helpful (building identifier)
- bbl, not helpful (block identifier)
- council district, equivalent to cncldist
- census tract, missing values and other location data
- problems, one-hot encoded in other values
- status, all are alive after dropping dead and stumps
- stump_diam, only applies to stumps
'''
tree_df_dropped = tree_df_clean.drop(columns=[
    'borocode', 'x_sp', 'y_sp', 'state', 'nta_name', 'zip_city', 'address',
    'spc_latin', 'created_at', 'tree_id', 'block_id', 'user_type', 'bin',
    'bbl', 'council district', 'boro_ct', 'census tract', 'problems', 'status',
    'stump_diam'
])

In [ ]:
# remianing NA values
vals = (tree_df_dropped.isna().sum() * 100 / len(tree_df_dropped)).values
keys = tree_df_dropped.columns
plt.tight_layout()
plt.bar(keys, vals)
plt.xlabel("Columns")
plt.ylabel("Percent missing")
plt.xticks(rotation=90, fontsize=8)
plt.title("Percent missing by column without dead trees and dropped columns")
plt.show()

In [ ]:
# drop those rows
tree_df_processed = tree_df_dropped.dropna(how='any')
print("Trees with missing values:", tree_df_dropped.shape[0] - tree_df_processed.shape[0])
miss_val = tree_df_processed.shape[0]

In [ ]:
# drop invalid zipcode
tree_df_processed = tree_df_processed[tree_df_processed['postcode'] != 83]

In [ ]:
# plot each quantitaive feature
vals = [
    'tree_dbh', 'postcode', 'community board', 'cncldist', 'st_assem',
    'st_senate', 'latitude', 'longitude'
]

fig, axs = plt.subplots(len(vals) // 2, 2, figsize=(10, 12), constrained_layout=True)

for i, val in enumerate(vals):
    axs[i // 2, i % 2].hist(tree_df_processed[val], bins=50)
    axs[i // 2, i % 2].set(xlabel=val, ylabel="Number of trees")

plt.show()

In [ ]:
# plot each categorical feature realtive to the health of the tree
def cat_plot(feature, ax):
    df = tree_df_processed[['health', feature]].value_counts().to_frame()
    df = df.unstack(level=0)
    df.columns = ['Fair', 'Good', 'Poor']
    df.plot(kind='bar', ax=ax)

vals = [
    'guards', 'curb_loc', 'steward', 'sidewalk', 'root_stone', 'root_grate',
    'root_other', 'trunk_wire', 'trnk_light', 'trnk_other', 'brch_light', 'brch_shoe',
    'brch_other', 'borough'
]
    
fig, axs = plt.subplots(len(vals) // 2, 2, figsize=(8, 18), constrained_layout=True)

for i, val in enumerate(vals):
    cat_plot(val, axs[i // 2, i % 2])

plt.show()

In [ ]:
# look for multicolinearity
# Ordinal encoding
tree_df_encoded = tree_df_processed.copy(deep=True)

health = ['Poor', 'Fair', 'Good']
enc = OrdinalEncoder(categories=[health])
tree_df_encoded['health'] = enc.fit_transform(tree_df_processed[['health']])

steward = ['None', '1or2', '3or4', '4orMore']
enc = OrdinalEncoder(categories=[steward])
tree_df_encoded['steward'] = enc.fit_transform(tree_df_encoded[['steward']])

sidewalk = ['NoDamage', 'Damage']
enc = OrdinalEncoder(categories=[sidewalk])
tree_df_encoded['sidewalk'] = enc.fit_transform(tree_df_encoded[['sidewalk']])

curbloc = ['OnCurb', 'OffsetFromCurb']
enc = OrdinalEncoder(categories=[curbloc])
tree_df_encoded['curb_loc'] = enc.fit_transform(tree_df_encoded[['curb_loc']])

yes_no = ['No', 'Yes']
enc = OrdinalEncoder(categories=[yes_no])

yes_no_features = [
    'brch_other', 'brch_shoe', 'brch_light', 'trnk_other', 'trnk_light',
    'trunk_wire', 'root_other', 'root_grate', 'root_stone'
]

for feat in yes_no_features:
    tree_df_encoded[feat] = enc.fit_transform(tree_df_encoded[[feat]])
    
# One-hot encode
enc = OneHotEncoder()
guards = enc.fit_transform(tree_df_encoded[['guards']])
tree_df_encoded['guards_' + enc.categories_[0][:]] = guards.toarray()
tree_df_encoded = tree_df_encoded.drop(columns=['guards'])

guards = enc.fit_transform(tree_df_encoded[['borough']])
tree_df_encoded['borough_' + enc.categories_[0][:]] = guards.toarray()
tree_df_encoded = tree_df_encoded.drop(columns=['borough'])

corr = tree_df_encoded.corr()

sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)
plt.show()

In [ ]:
# check species feature for usability
tree_df_encoded['spc_common'].value_counts()

## Preprocessing

In [ ]:
# load in data with relative path
tree_df = pd.read_csv('2015_Street_Tree_Census_-_Tree_Data.csv')

In [ ]:
# drop columns
tree_df = tree_df.drop(columns=[
    'borocode', 'x_sp', 'y_sp', 'state', 'nta_name', 'zip_city', 'address',
    'spc_latin', 'created_at', 'tree_id', 'block_id', 'user_type', 'bin',
    'bbl', 'council district', 'boro_ct', 'census tract', 'problems', 'status',
    'stump_diam', 'postcode', 'community board', 'cncldist', 'st_assem', 'st_senate', 
    'nta', 'spc_common'
])

In [ ]:
# drop NA and dead tree values
tree_df = tree_df[tree_df['health'].notna()]
tree_df = tree_df.dropna(how='any')

In [ ]:
# scale diameters 
scaler = StandardScaler()
tree_df['tree_dbh'] = scaler.fit_transform(tree_df[['tree_dbh']])

In [ ]:
# Ordinal Encode
health = ['Poor', 'Fair', 'Good']
enc = OrdinalEncoder(categories=[health])
tree_df['health'] = enc.fit_transform(tree_df[['health']])

steward = ['None', '1or2', '3or4', '4orMore']
enc = OrdinalEncoder(categories=[steward])
tree_df['steward'] = enc.fit_transform(tree_df[['steward']])

sidewalk = ['NoDamage', 'Damage']
enc = OrdinalEncoder(categories=[sidewalk])
tree_df['sidewalk'] = enc.fit_transform(tree_df[['sidewalk']])

curbloc = ['OnCurb', 'OffsetFromCurb']
enc = OrdinalEncoder(categories=[curbloc])
tree_df['curb_loc'] = enc.fit_transform(tree_df[['curb_loc']])

yes_no = ['No', 'Yes']
enc = OrdinalEncoder(categories=[yes_no])

yes_no_features = [
    'brch_other', 'brch_shoe', 'brch_light', 'trnk_other', 'trnk_light',
    'trunk_wire', 'root_other', 'root_grate', 'root_stone'
]

for feat in yes_no_features:
    tree_df[feat] = enc.fit_transform(tree_df[[feat]])
    
# One Hot Encode
enc = OneHotEncoder()
guards = enc.fit_transform(tree_df[['guards']])
tree_df['guards_' + enc.categories_[0][:]] = guards.toarray()
tree_df = tree_df.drop(columns=['guards'])

guards = enc.fit_transform(tree_df[['borough']])
tree_df['borough_' + enc.categories_[0][:]] = guards.toarray()
tree_df = tree_df.drop(columns=['borough'])

In [ ]:
# split to x and y sets
y = np.asarray(tree_df['health'])
x = tree_df.drop(columns=['health'])

In [ ]:
# random sample (stratified) for dev, test, train and val
x_dev, x_test, y_dev, y_test = train_test_split(x, y, test_size=0.2, random_state=42, stratify = y)
x_train, x_val, y_train, y_val = train_test_split(x_dev, y_dev, test_size=0.2, random_state=42, stratify = y_dev)

In [ ]:
# over-sampling of training data
ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(x_train, y_train)

In [ ]:
# under sampling of training data
rus = RandomUnderSampler(random_state=42)
x_rus, y_rus = rus.fit_resample(x_train, y_train)

In [ ]:
# smote sampling of training data
smote = SMOTE(random_state=42)
x_smote, y_smote = smote.fit_resample(x_train, y_train)

## Logistic Regression

## Decision Tree

In [ ]:
def model_result(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train)
    y_train_predict = model.predict(x_train)
    y_test_predict = model.predict(x_test)    
    # accuracy scores
    acc_train = accuracy_score(y_train,y_train_predict)
    acc_test = accuracy_score(y_test,y_test_predict)
    # print trainind and testing accuracy
    print("Accuracy of Training data: ", acc_train)
    print("Accuracy of Testing data: ", acc_test)
    # print train and test f1 score
    f1_train = f1_score(y_train, y_train_predict , average='weighted')
    f1_test = f1_score(y_test, y_test_predict , average='weighted')
    print("weigted f1 of Training data: ", f1_train)
    print("weigted f1 of Testing data: ", f1_test)

    # confusion_matrix
    cf = confusion_matrix(y_test, model.predict(x_test))
    sns.heatmap(cf, annot = True, fmt = 'g')
    plt.show()

In [ ]:
def dt_select(x_train,y_train,score):
    param_grid = {
        'max_depth': [4,8,10,12,15,20],
        'min_samples_split': np.linspace(0.1, 0.3, 3, endpoint=True),
        "max_features":["log2","sqrt",None],
        "max_leaf_nodes": [None,10,20,30],
        'ccp_alpha' : np.logspace(-6, -4, 3)
    }
    # Create a based model
    dt = DecisionTreeClassifier(random_state=100)

    # Instantiate the random search model
    randomized_search = RandomizedSearchCV(estimator = dt, param_distributions = param_grid, scoring=score,cv=5)

    randomized_model=randomized_search.fit(x_train,y_train)
    return randomized_model

In [ ]:
# random stratified sampling
# accuracy as scoring metric
random_best = dt_select(x_train,y_train,'accuracy')
print("optimal hyperparameters",random_best.best_params_)
print("optimal accuracy score", random_best.best_score_)
print("test accuracy score",random_best.score(x_test,y_test))
#f1 as metric for hyperparamter tuning
model_result(random_best, x_train, y_train, x_test, y_test)
#f1 as scoring metric
random_best_f1 = dt_select(x_train,y_train,'f1_weighted')
print("optimal hyperparameters",random_best_f1.best_params_)
print("optimal f1 score", random_best_f1.best_score_)
print("test f1 score",random_best_f1.score(x_test,y_test))
#f1 as metric for hyperparamter tuning
model_result(random_best_f1, x_train, y_train, x_test, y_test)

In [ ]:
# random over sampling
# accuracy as scoring metric
over_best = dt_select(x_ros, y_ros,'accuracy')
print("optimal hyperparameters",over_best.best_params_)
print("optimal accuracy score", over_best.best_score_)
print("test accuracy",over_best.score(x_test,y_test))
#accuracy as metric for hyperparamter tuning
model_result(over_best, x_ros, y_ros, x_test, y_test)
# f1 as score metric
over_best_f1 = dt_select(x_ros, y_ros,'f1_weighted')
print("optimal hyperparameters",over_best_f1.best_params_)
print("optimal accuracy score", over_best_f1.best_score_)
print("test accuracy",over_best_f1.score(x_test,y_test))
#f1 as metric for hyperparamter tuning
model_result(over_best_f1, x_ros, y_ros, x_test, y_test)

In [ ]:
# random under sampling
# accuracy as scoring metric
under_best = dt_select(x_rus, y_rus,'accuracy')
print("optimal hyperparameters",under_best.best_params_)
print("optimal accuracy score", under_best.best_score_)
print("test accuracy",under_best.score(x_test,y_test))
# accuracy as metric for hyperparamter tuning
model_result(under_best, x_rus, y_rus, x_test, y_test)
# f1 as score metric
under_best_f1 = dt_select(x_rus, y_rus,'f1_weighted')
print("optimal hyperparameters",under_best_f1.best_params_)
print("optimal f1 score", under_best_f1.best_score_)
print("test f1 score",under_best_f1.score(x_test,y_test))
# f1 as metric for hyperparamter pruning
model_result(under_best_f1, x_rus, y_rus, x_test, y_test)

In [ ]:
# smote sampling
# accuracy as metric
smote_best = dt_select(x_smote, y_smote,"accuracy")
print("optimal hyperparameters",smote_best.best_params_)
print("optimal accuracy score", smote_best.best_score_)
print("test accuracy",smote_best.score(x_test,y_test))
#accuracy as metric for hyperparamter pruning
model_result(smote_best, x_smote, y_smote, x_test, y_test)
# f1 sccore as metric
smote_best_f1 = dt_select(x_smote, y_smote,'f1_weighted')
print("optimal hyperparameters",smote_best_f1.best_params_)
print("optimal f1 score", smote_best_f1.best_score_)
print("test f1 score",smote_best_f1.score(x_test,y_test))
# f1 as metric for hyperparamter pruning
model_result(smote_best_f1, x_smote, y_smote, x_test, y_test)

## Support Vector Machine

In [ ]:
def SVM_result(svm, X_train, y_train, X_test, y_test, report = False, cf = False):
    # fit the model
    svm.fit(X_train, y_train)
    # predict y
    y_train_predict = svm.predict(X_train)
    y_test_predict = svm.predict(X_test)    
    # accuracy scores
    acc_predit_train = accuracy_score(y_train_predict,y_train)
    acc_predit_test = accuracy_score(y_test_predict,y_test)
    # print trainind and testing accuracy
    print("Accuracy of Training data: ", acc_predit_train)
    print("Accuracy of Testing data: ", acc_predit_test)
    # show result report
    if (report):
        print(classification_report(y_test, y_test_predict))
    # show confusion_matrix
    if (cf):
        cf = confusion_matrix(y_test, primal_svm.predict(X_test))
        sns.heatmap(cf, annot = True, fmt = 'g')

In [ ]:
# Basic SVM models
import warnings
warnings.filterwarnings('ignore')
# basic primal and dual svm before hyperparameter tunin
svm = SVC(random_state=100)
# random search for hyperparameter tuning
param_list = {'C': stats.uniform(0, 5), 
              'kernel': ['linear', 'poly', 'sigmoid', 'rbf'], 
              'gamma': [.01, 1, 10, 100]} # random list for hyperparameter tuning
rand_search = RandomizedSearchCV(svm, 
                                 param_distributions = param_list, 
                                 cv = 5, 
                                 random_state = 2022) 

In [ ]:
# Hyperparameter tuning random Sampling
rand_search.fit(x_train, y_train) 
svm_best = rand_search.best_estimator_
print(svm_best)
SVM_result(svm_best, x_train, y_train, x_test, y_test)

In [ ]:
# Hyperparameter tuning under sampling
rand_search.fit(x_rus, y_rus) 
svm_best = rand_search.best_estimator_
print(svm_best)
SVM_result(svm_best, x_rus, y_rus, x_test, y_test)

In [ ]:
rand_search.fit(x_ros, y_ros)
svm_best = rand_search.best_estimator_
print(svm_best)
SVM_result(svm_best, x_ros, y_ros, x_test, y_test, cf = False)

In [ ]:
rand_search.fit(x_smote, y_smote)
svm_best = rand_search.best_estimator_
print(svm_best)
SVM_result(svm_best, x_smote, y_smote, x_test, y_test, cf = False)

## Tal